In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv("../Data/Biometric Data Analysis/train.csv").fillna(0)

In [3]:
data.rho.unique()
data.drop("id", axis=1, inplace=True)

In [4]:
# group by rho

data_rho_25 = data[data['rho'] == 25]
data_rho_20 = data[data['rho'] == 20]
data_rho_15 = data[data['rho'] == 15]
data_rho_10 = data[data['rho'] == 10]

In [5]:
data_rho_25.to_csv("data/data_rho_25.csv")
data_rho_20.to_csv("data/data_rho_20.csv")
data_rho_15.to_csv("data/data_rho_15.csv")
data_rho_10.to_csv("data/data_rho_10.csv")

In [6]:
data_rho_25.iloc[:,1:-4].head()

,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,740_src,...,900_dst,910_dst,920_dst,930_dst,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst
0,0.37950,0.42993,0.52076,0.57166,0.67818,0.75476,0.83580,0.93623,0.96333,1.01541,...,0.000000e+00,3.527371e-18,0.000000e+00,6.455564e-19,0.000000e+00,0.000000e+00,0.000000e+00,1.067504e-18,5.998949e-18,4.378513e-17
2,0.00000,0.03289,0.02416,0.03610,0.05843,0.09015,0.14944,0.18578,0.25584,0.30445,...,4.629125e-18,1.409413e-18,3.237480e-18,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.329725e-18,0.000000e+00
5,0.02976,0.03531,0.01180,0.03343,0.04524,0.03576,0.05347,0.02968,0.04358,0.05270,...,0.000000e+00,3.749523e-18,0.000000e+00,1.076966e-18,1.820508e-18,2.682566e-18,0.000000e+00,0.000000e+00,8.175377e-18,7.686989e-17
7,0.03749,0.03493,0.07953,0.09259,0.14327,0.21181,0.27927,0.35796,0.46093,0.53271,...,4.488662e-19,9.023918e-19,1.273972e-17,1.164347e-18,0.000000e+00,1.846159e-18,2.910492e-18,7.935297e-19,0.000000e+00,0.000000e+00
9,0.00000,0.00084,0.00326,0.00000,0.02250,0.01376,0.03809,0.06749,0.11427,0.17416,...,3.552491e-19,0.000000e+00,5.516710e-20,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.359810e-19,3.818551e-19,0.000000e+00


In [7]:
data_rho_25.iloc[:,1:-4] = data_rho_25.iloc[:,1:-4] * (25**4)
data_rho_20.iloc[:,1:-4] = data_rho_25.iloc[:,1:-4] * (20**4)
data_rho_15.iloc[:,1:-4] = data_rho_25.iloc[:,1:-4] * (15**4)
data_rho_10.iloc[:,1:-4] = data_rho_25.iloc[:,1:-4] * (10**4)

C:\Users\lonj7\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### XG Boost Classifier

In [8]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [9]:
def XY(df):
    X = df.iloc[:,:-4]
    y_hhb = df.iloc[:,-4]
    y_hhbo2 = df.iloc[:,-3]
    y_ca = df.iloc[:,-2]
    y_na = df.iloc[:,-1]
    
    return X, y_hhb, y_hhbo2, y_ca, y_na

In [10]:
def XG_Boost(df, num):
    X, y_hhb, y_hhbo2, y_ca, y_na = XY(df)
    
    # Split data
    X1_train, X1_test, y1_train, y1_test = train_test_split(X, y_hhb, test_size=0.2, shuffle=1234)
    X2_train, X2_test, y2_train, y2_test = train_test_split(X, y_hhbo2, test_size=0.2, shuffle=1234)
    X3_train, X3_test, y3_train, y3_test = train_test_split(X, y_ca, test_size=0.2, shuffle=1234)
    X4_train, X4_test, y4_train, y4_test = train_test_split(X, y_na, test_size=0.2, shuffle=1234)
    
    
    xg_reg_hbb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
    xg_reg_hbo2 = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
    xg_reg_ca = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
    xg_reg_na = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
    
    
    xg_reg_hbb.fit(X1_train,y1_train)
    preds_hbb = xg_reg_hbb.predict(X1_test)

    xg_reg_hbo2.fit(X2_train,y2_train)
    preds_hbo2 = xg_reg_hbo2.predict(X2_test)

    xg_reg_ca.fit(X3_train,y3_train)
    preds_ca = xg_reg_ca.predict(X3_test)

    xg_reg_na.fit(X4_train,y4_train)
    preds_na = xg_reg_na.predict(X4_test)
    
    
    rmse_hbb = np.sqrt(mean_squared_error(y1_test, preds_hbb))
    rmse_hbo2 = np.sqrt(mean_squared_error(y2_test, preds_hbo2))
    rmse_ca = np.sqrt(mean_squared_error(y3_test, preds_ca))
    rmse_na = np.sqrt(mean_squared_error(y4_test, preds_na))

    print(num, " mm")
    print("RMSE - hbb  : %f" % (rmse_hbb))
    print("RMSE - hbo2 : %f" % (rmse_hbo2))
    print("RMSE - ca   : %f" % (rmse_ca))
    print("RMSE - na   : %f" % (rmse_na))
    
    
    return xg_reg_hbb, xg_reg_hbo2, xg_reg_ca, xg_reg_na

In [11]:
# data_rho_25, data_rho_20, data_rho_15, data_rho_10

hbb_25, hbo2_25, ca_25, na_25 = XG_Boost(data_rho_25, 25)

hbb_20, hbo2_20, ca_20, na_20 = XG_Boost(data_rho_20, 20)

hbb_15, hbo2_15, ca_15, na_15 = XG_Boost(data_rho_15, 15)

hbb_10, hbo2_10, ca_10, na_10 = XG_Boost(data_rho_10, 10)

25  mm
RMSE - hbb  : 1.440496
RMSE - hbo2 : 0.809499
RMSE - ca   : 2.734549
RMSE - na   : 1.960708
20  mm
RMSE - hbb  : 2.951132
RMSE - hbo2 : 0.995797
RMSE - ca   : 2.968019
RMSE - na   : 1.910564
15  mm
RMSE - hbb  : 3.110037
RMSE - hbo2 : 1.066527
RMSE - ca   : 2.972334
RMSE - na   : 1.889767
10  mm
RMSE - hbb  : 3.007669
RMSE - hbo2 : 0.993488
RMSE - ca   : 2.921155
RMSE - na   : 1.814702


In [23]:
test = pd.read_csv("../Data/Biometric Data Analysis/test.csv").fillna(0)

In [24]:
test_rho_25 = test[test['rho'] == 25]
test_rho_20 = test[test['rho'] == 20]
test_rho_15 = test[test['rho'] == 15]
test_rho_10 = test[test['rho'] == 10]

In [25]:
test_rho_25.to_csv("data/test_rho_25.csv")
test_rho_20.to_csv("data/test_rho_20.csv")
test_rho_15.to_csv("data/test_rho_15.csv")
test_rho_10.to_csv("data/test_rho_10.csv")

In [26]:
def prediction(hbb, hbo2, ca, na, df):
    
    test = df.iloc[:, 1:]
    
    preds_hbb = hbb.predict(test)
    preds_hbo2 = hbo2.predict(test)
    preds_ca = ca.predict(test)
    preds_na = na.predict(test)
    
    df_1 = df.copy()
    
    df_1['hbb'] = preds_hbb
    df_1['hbo2'] = preds_hbo2
    df_1['ca'] = preds_ca
    df_1['na'] = preds_na
    
    return df_1

In [27]:
# test_rho_25 - hbb_25, hbo2_25, ca_25, na_25
# test_rho_20 - hbb_20, hbo2_20, ca_20, na_20
# test_rho_15 - hbb_15, hbo2_15, ca_15, na_15
# test_rho_10 - hbb_10, hbo2_10, ca_10, na_10

rho_25 = prediction(hbb_25, hbo2_25, ca_25, na_25, test_rho_25)
rho_20 = prediction(hbb_20, hbo2_20, ca_20, na_20, test_rho_20)
rho_15 = prediction(hbb_15, hbo2_15, ca_15, na_15, test_rho_15)
rho_10 = prediction(hbb_10, hbo2_10, ca_10, na_10, test_rho_10)

In [28]:
temp = pd.concat([rho_25, rho_20, rho_15, rho_10]).sort_values('id')
id = temp.id
temp = temp.iloc[:, -4:]

In [29]:
submission = pd.concat([id, temp], axis=1)
submission.to_csv("submission/07.csv")

10000